In [1]:
import os, re, subprocess, urllib2

In [2]:
def test_thumbnail(spec, reference_name):
    computed_dir = 'temp-computed'
    try:
        os.mkdir(computed_dir)
    except OSError:
        pass

    computed_path = computed_dir + '/' + reference_name
    reference_path = 'reference/' + reference_name

    print 'Comparing %s with %s...' % (reference_path, spec)
    thumbnail_cmd = ['../thumbnail-server.rb', spec]
    computed_image = subprocess.check_output(thumbnail_cmd)

    open(computed_path,'wb').write(computed_image)
    # Is it really an image?
    if subprocess.call(['identify', computed_path]):
        print 'Error creating thumbnail (%s):' % (' '.join(thumbnail_cmd))
        print computed_image
        return False

    if os.path.exists(reference_path):
        command = ['compare', '-metric', 'RMSE', computed_path, reference_path, 'null:']
        p = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        (stdout, stderr) = p.communicate()
        status = p.wait()
        rmse = float(re.search(r'\((.*)\)', stderr).group(1))
        if rmse > 0.001:
            print 'Failed with rmse = %g' % rmse
            return False
        print 'Succeeded with rmse = %g' % rmse
        return True
    else:
        print 'Failed because %s does not exist.  If you want to copy computed to it, type' % reference_path
        print 'mv %s %s' % (computed_path, reference_path)
        return False

In [3]:
def test_vegas(frametime, frameno):
    spec = 'root=http://earthengine.google.org/timelapse/data/20130507/&boundsLTRB=239789.4575441497,523876.80428915547,240835.17182986398,524469.1852415365&width=640&frameTime=%g&format=png' % frametime
    reference_name = 'vegas%02d.png' % frameno
    test_thumbnail(spec, reference_name)

# Test beginning of video
test_vegas(frametime = 0.0, frameno = 0)
test_vegas(frametime = 0.09, frameno = 0)

# Test centers of frames 0 to 28, inclusive
for frameno in range(0, 29):
    test_vegas(frametime=(frameno * 0.1 + 0.05), frameno=frameno)
    
# Test end of video
test_vegas(frametime = 2.8, frameno = 28)
test_vegas(frametime = 2.89, frameno = 28)

Comparing reference/vegas00.png with root=http://earthengine.google.org/timelapse/data/20130507/&boundsLTRB=239789.4575441497,523876.80428915547,240835.17182986398,524469.1852415365&width=640&frameTime=0&format=png...


OSError: [Errno 2] No such file or directory